In [19]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import pandas as pd
import math
import numpy as np
import os
import time 

import cntk as C
import cntk.tests.test_utils
import pickle
import random
from cntk import sequence
from cntk import load_model
from cntk.device import try_set_default_device, gpu,cpu
from scipy.sparse import csr_matrix

cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components
try_set_default_device(gpu(0))


vocab_size = 80000
num_labels = 19
title_size = 52000
body_size  = 210000
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200

max_length_title = 30
max_length_body  = 100



suffix = "180days_sample"
prefix = "/home/t-haohu/IndustryClassifier/Data/"

data_token_body        = "{}/middle/{}_token_body.txt".format(prefix,suffix)
data_train_sample_body = "{}/middle/train_{}_body.txt".format(prefix,suffix)
data_test_sample_body  = "{}/middle/test_{}_body.txt".format(prefix,suffix)

data_industry_sample = "{}/ready/industry_{}.wl".format(prefix,suffix)
data_title_sample    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_sample     = "{}/ready/body_{}.wl".format(prefix,suffix)

 

def load_data_body(input_file,title_dict,body_dict,industry_dict):
    data = open(input_file, encoding = "utf-8").readlines()
    
    data_title = np.zeros((len(data),max_length_title),dtype = np.float32)
    data_body  = np.zeros((len(data),max_length_body),dtype = np.float32)
    data_label = np.zeros((len(data),1),dtype = np.float32)
    
    
    for index,line in enumerate(data):
        row = line.strip("\n").split("\t")
        
        title    =  row[0]
        body     =  row[1]
        industry =  row[2]
        
        for jndex,token in enumerate(title.split(" ")):
            if jndex>=max_length_title:
                break
            data_title[index,jndex]=title_dict.get(token,len(title_dict)-1)
            
        for jndex,token in enumerate(body.split(" ")):
            if jndex>=max_length_body:
                break
            data_body[index,jndex]=body_dict.get(token,len(title_dict)-1)
            
        data_label[index] = industry_dict.get(industry,len(industry_dict))
    return data_title,data_body,data_label

def create_model_cnn_with_body():
    
    h1t= C.layers.Embedding(300,name='embed')(input_xt_one_hot)#init=embedding,
    h1b= C.layers.Embedding(300,name='embed')(input_xb_one_hot)#init=embedding,
    
    #bnb = C.layers.BatchNormalization(name='bn')(h1b)
    #bnt = C.layers.BatchNormalization(name='bn')(h1t)



    h2_1t=C.layers.Convolution((1,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1t)
    h2_2t=C.layers.Convolution((2,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1t)
    h2_3t=C.layers.Convolution((3,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1t)

    h2_1b=C.layers.Convolution((1,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1b)
    h2_2b=C.layers.Convolution((2,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1b)
    h2_3b=C.layers.Convolution((3,emb_dim),num_filters=50,reduction_rank=0,activation=C.relu)(h1b)

    h3_2t=C.layers.MaxPooling((max_length_title-1,1),name='pooling_t_1')(h2_2t)
    h3_1t=C.layers.MaxPooling((max_length_title-0,1),name='pooling_t_2')(h2_1t)
    h3_3t=C.layers.MaxPooling((max_length_title-2,1),name='pooling_t_3')(h2_3t)

    h3_2b=C.layers.MaxPooling((max_length_body-1,1),name='pooling_b_1')(h2_2b)
    h3_1b=C.layers.MaxPooling((max_length_body-0,1),name='pooling_b_2')(h2_1b)
    h3_3b=C.layers.MaxPooling((max_length_body-2,1),name='pooling_b_3')(h2_3b)

    h3=C.splice(h3_2t,h3_1t,h3_3t,h3_2b,h3_1b,h3_3b,axis=0)

    #h4=C.layers.Dense(hidden_dim, activation=C.relu,name='hidden')(h3)
    #drop1 = C.layers.Dropout(0.5,name='drop1')(h3)

    h4=C.layers.Dense(num_labels,name='classify')(h3)

    return h4

def batch_iter(data,batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    print('data_size: ', data_size, 'batch_size: ', batch_size, 'num_batches_per_epoch: ', num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            random.shuffle(data)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[start_index:end_index]
            

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_sample).readlines()])}
body_dict  =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_body_sample ).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_sample).readlines()])}




input_xt = C.input_variable(shape=(max_length_title),  dtype=np.float32)
input_xb = C.input_variable(shape=(max_length_body) ,  dtype=np.float32)
input_y  = C.input_variable(shape=(1)               ,  dtype=np.int)

input_xt_one_hot = C.one_hot(input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
input_xb_one_hot = C.one_hot(input_xb, num_classes=len(body_dict)    ,  sparse_output=True) 
input_y_one_hot = C.one_hot(input_y  , num_classes=len(industry_dict) ,  sparse_output=True)

#test_data_title, test_data_body, test_data_label  = load_data_body(data_test_sample_body,title_dict,body_dict,industry_dict)
#train_data_title,train_data_body,train_data_label = load_data_body(data_train_sample_body,title_dict,body_dict,industry_dict)

test_data  = load_data_body(data_test_sample_body,title_dict,body_dict,industry_dict)
train_data = load_data_body(data_train_sample_body,title_dict,body_dict,industry_dict)

def test_body(batch_size,model,data):
    scores = model(input_xt,input_xb)
    predict = C.argmax(scores,axis = 0)
    confuse = np.zeros((num_labels,num_labels))
    #C.element_add(input_y,C.element_times(predict,C.Constant([nums_labels])))
    test_data_title,test_data_body,test_data_label = data
    batches = batch_iter(list(zip(test_data_title,test_data_body,test_data_label)), batch_size, 1)
    
    for batch in batches:
        batch_data_title,batch_data_body,batch_data_label = zip(*batch) 
        output = np.array(predict.eval({input_xb: np.array(batch_data_body),input_xt: np.array(batch_data_title)}),dtype=np.int)
        gt = np.array(batch_data_label,dtype=np.int)
        confuse+=fast_hist(output,gt,num_labels)
    precision=np.diag(confuse)/np.sum(confuse,axis=0)
    recall = np.diag(confuse)/np.sum(confuse,axis=1)
    accuarcy = np.diag(confuse).sum() / confuse.sum()
    aver_precision=np.nanmean(precision)
    aver_recall = np.nanmean(recall)
   
    print("Precision:{} Recall:{} Acc:{}".format(aver_precision,aver_recall,accuarcy))
def train_body(train_data,num_epochs,learning_rate,batch_size,l2_weight=0):
    
    #learning_rate *= batch_size
    model = create_model_cnn_with_body()
    print(C.logging.get_node_outputs(model))
    scores = model(input_xt,input_xb)

    loss =C.reduce_mean(C.losses.cross_entropy_with_softmax(scores, input_y_one_hot))
    
    # Training
    lr_schedule = C.learning_parameter_schedule(learning_rate)
    #learner = C.adam(scores.parameters, lr=lr_schedule, momentum=0.9,l2_regularization_weight=0)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    momentums = C.momentum_schedule(0.99, minibatch_size=batch_size)
    learner = C.adam(parameters=scores.parameters,#model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15,
                     gradient_clipping_with_truncation=True,
                     l2_regularization_weight=l2_weight)
    trainer = C.Trainer(scores, (loss), [learner], progress_printer)
    
    train_data_title,train_data_body,train_data_label = train_data
    batches = batch_iter(list(zip(train_data_title,train_data_body,train_data_label)), batch_size, num_epochs)

    # training loop
    count = 0
    t = time.time()
    for batch in batches:
        count += 1
        batch_data_title,batch_data_body,batch_data_label = zip(*batch)
        #print(np.array(batch_data_body).shape)
        #batch_data_title,batch_data_body,batch_data_label = transfer(batch_data_title),transfer(batch_data_body),trainsfer(batch_data_label)
        trainer.train_minibatch({input_xb: np.array(batch_data_body),input_xt: np.array(batch_data_title), input_y: np.array(batch_data_label)})
        if count%1000== 0:
            print(count,time.time()-t)
            t=time.time()
            test(batch_size,model,(data_test_sample_body ,title_dict,body_dict,industry_dict))

    # save model
    model.save('./model/IndustryClassifyCNN_body.dnn')


In [20]:
train(num_epochs=20,learning_rate=[1e-3*30]*2+[1e-4*30],batch_size = 42)

[Output('classify', [#], [19]), Output('Splice62824_Output_0', [#], [300 x 1 x 1]), Output('pooling_t_1', [#], [50 x 1 x 1]), Output('Block62303_Output_0', [#], [50 x 29 x 1]), Output('embed', [#], [30 x 300]), Output('OneHotOp62135_Output_0', [#], [30 x 23184]), Output('pooling_t_2', [#], [50 x 1 x 1]), Output('Block62240_Output_0', [#], [50 x 30 x 1]), Output('pooling_t_3', [#], [50 x 1 x 1]), Output('Block62366_Output_0', [#], [50 x 28 x 1]), Output('pooling_b_1', [#], [50 x 1 x 1]), Output('Block62492_Output_0', [#], [50 x 99 x 1]), Output('embed', [#], [100 x 300]), Output('OneHotOp62138_Output_0', [#], [100 x 60491]), Output('pooling_b_2', [#], [50 x 1 x 1]), Output('Block62429_Output_0', [#], [50 x 100 x 1]), Output('pooling_b_3', [#], [50 x 1 x 1]), Output('Block62555_Output_0', [#], [50 x 98 x 1])]
data_size:  41940 batch_size:  30 num_batches_per_epoch:  1398
Learning rate per minibatch: 0.03
1000 36.22497200965881
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600

/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/ipykernel_launcher.py:159: RuntimeWarning: invalid value encountered in true_divide


Precision:0.5324430853511949 Recall:0.6902382816627641 Acc:0.704144645340751
2000 40.1063814163208
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6264493572648457 Recall:0.6826649727497918 Acc:0.7337969401947149
3000 40.165118932724
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6447254218870008 Recall:0.6904095389824263 Acc:0.7491515994436717
4000 40.35217499732971
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6434215014286137 Recall:0.6714896054994925 Acc:0.7363004172461752
5000 40.23284864425659
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6483243291678749 Recall:0.6697220329971958 Acc:0.7425312934631433
6000 40.12236523628235
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6447874827329285 Recall:0.6747400128822413 Acc:0.7453685674547983
7000 39.97572350502014
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.64685920089809

In [16]:
x = [1,2,3]
y = [4,5,6]
zipped = zip(x,y)
unzipped_x, unzipped_y = zip(*zipped)
print(list(zipped),unzipped_x)
txy=(x,y)
a,b =txy
print(a,b)

[] (1, 2, 3)
[1, 2, 3] [4, 5, 6]


In [75]:
C.cntk_py.set_fixed_random_seed(int(time.time()))
W = C.Constant([[1,1],[1,1]],dtype=np.float64)
x = C.input_variable(shape=(2,1),dtype=np.float64)
h1 = C.times(W,x)
h2=C.layers.Dropout(0.5)(h1)
h3 = C.times(W,h1)
h4 = C.times(W,h2)
print(h3.eval({x:np.array([[2],[1]],dtype=np.float64)}))
print(h4.eval({x:np.array([[2],[1]],dtype=np.float64)}))

[[[6.]
  [6.]]]
[[[6.]
  [6.]]]
